In [1]:
import os
import h5py
import numpy as np
from IPython.lib.deepreload import reload
import matplotlib.pyplot as plt
from collections import OrderedDict
import skimage.io as io
from tqdm import tqdm
%matplotlib qt

In [ ]:
all_hkls, all_qs, all_fs = generate_reciprocal_lattice(test.phases['sapphire'], tth_range=(0, 60))
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

plot_qs = euler_rotation(all_qs, -60, 0, 0)
ax.scatter(*np.asarray(plot_qs).T, c='k', s=all_fs/20)
#ax.scatter(*np.asarray(all_qs).T, c='k', s=all_fs/20)
ax.scatter(0, 0, 0, c='k', s=10)

ax.scatter(*np.asarray(plot_qs)[13], c='r', s=10)

for theta in [0, 5, 5.5, 12.25, 13.625, 14.20, 15.55]:
    theta = np.radians(theta)

    q = get_q_vect(test.tth_arr, test.chi_arr, wavelength=energy_2_wavelength(18))

    Ry = np.array([[np.cos(theta), 0, -np.sin(theta)],
                   [0, 1, 0],
                   [np.sin(theta), 0, np.cos(theta)]])

    # Plot sampled Ewald sphere
    q_mask = q[:, test.map.mask]

    q_mask = (q_mask.T @ Ry).T
    ax.plot_trisurf(q_mask[0].ravel()[::skip],
                    q_mask[1].ravel()[::skip],
                    q_mask[2].ravel()[::skip],
                    alpha=0.5, label='detector')

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')

fig.show()

In [ ]:
are_coplanar([[1, 0, 0], [2, 0, 0], [1, 1, 0]])

True

In [ ]:
pixel_indices = [0, 0]
skip = 500

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

plot_qs = euler_rotation(all_qs, 10, -20, 0)

ax.scatter(*np.asarray(plot_qs).T, c='k', s=all_fs / 50, alpha=0.75)
ax.scatter(0, 0, 0, c='k', s=10)

# Plot full Ewald sphere
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
radius = 2 * np.pi / test.wavelength
x =  radius * np.outer(np.cos(u), np.sin(v))
y = radius * np.outer(np.sin(u), np.sin(v))
z = radius * np.outer(np.ones(np.size(u)), np.cos(v))
#ax.plot_surface(x, y, z - radius, alpha=0.2, color='k', label='Ewald sphere')

q = get_q_vect(test.tth_arr, test.chi_arr, wavelength=test.wavelength)

if pixel_indices is not None:
    pixel_df = test.spots[(test.spots['map_x'] == pixel_indices[0])
                            & (test.spots['map_y'] == pixel_indices[1])].copy()


if pixel_indices is not None:
    ax.scatter(*pixel_df[['qx', 'qy', 'qz']].values.T, s=1, c='r', label='spots')

# Sample geometry
ax.quiver([0, 0], [0, 0], [-2 * radius, -radius], [0, 0], [0, 0], [radius, radius], colors='k')
ax.scatter(0, 0, 0, marker='o', s=10, facecolors='none', edgecolors='k', label='transmission')
ax.scatter(0, 0, -radius, marker='h', s=10, c='b', label='sample')

# Plot sampled Ewald sphere
q_mask = q[:, test.map.mask]
ax.plot_trisurf(q_mask[0].ravel()[::skip],
                q_mask[1].ravel()[::skip],
                q_mask[2].ravel()[::skip],
                alpha=0.5, label='detector')

ax.scatter(qx, qy, qz, c='b', s=1, alpha=0.1)

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')

fig.show()

In [ ]:
pixel_indices = [0, 0] #[26, 21]
skip = 500

#fig, ax = plt.subplots(1, 1, figsize=(5, 10), dpi=200, subplot_kw={'projection':'3d'})
fig = plt.figure(figsize=(10, 5), dpi=200)
ax = fig.add_axes([0, 0, 0.5, 1], projection='3d')
#ax = fig.add_subplot(projection='3d')

plot_qs = euler_rotation(all_qs, 10, -20, 0)
recip_latt = ax.scatter(*np.asarray(plot_qs).T, c='k', s=all_fs / 50, alpha=0.75)
ax.scatter(0, 0, 0, c='k', s=10)

# Plot full Ewald sphere
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
radius = 2 * np.pi / test.wavelength
x =  radius * np.outer(np.cos(u), np.sin(v))
y = radius * np.outer(np.sin(u), np.sin(v))
z = radius * np.outer(np.ones(np.size(u)), np.cos(v))
#ax.plot_surface(x, y, z - radius, alpha=0.2, color='k', label='Ewald sphere')

q = get_q_vect(test.tth_arr, test.chi_arr, wavelength=test.wavelength)

if pixel_indices is not None:
    pixel_df = test.spots[(test.spots['map_x'] == pixel_indices[0])
                            & (test.spots['map_y'] == pixel_indices[1])].copy()


if pixel_indices is not None:
    ax.scatter(*pixel_df[['qx', 'qy', 'qz']].values.T, s=1, c='r', label='spots')

# Sample geometry
ax.quiver([0, 0], [0, 0], [-2 * radius, -radius], [0, 0], [0, 0], [radius, radius], colors='k')
ax.scatter(0, 0, 0, marker='o', s=10, facecolors='none', edgecolors='k', label='transmission')
ax.scatter(0, 0, -radius, marker='h', s=10, c='b', label='sample')

# Plot sampled Ewald sphere
q_mask = q[:, test.map.mask]
ax.plot_trisurf(q_mask[0].ravel()[::skip],
                q_mask[1].ravel()[::skip],
                q_mask[2].ravel()[::skip],
                alpha=0.5, label='detector')

phi1, PHI, phi2 = 10, -20, 0
euler_angles = {'phi1' : phi1,
                'PHI' : PHI,
                'phi2': phi2}
euler_bounds = [[-180, 180], [0, 180], [-180, 180]]
slider_lst = []
update_lst = []

slider_vpos = np.linspace(0.8, 0.1, 3)

for i, key in enumerate(euler_angles.keys()):
    slider_ax = fig.add_axes

    # Make a horizontal slider to control the frequency.
    axfreq = fig.add_axes([0.7, slider_vpos[i], 0.2, 0.03])
    euler_slider = Slider(
        ax=axfreq,
        label=f'{key} [deg]',
        valmin=euler_bounds[i][0],
        valmax=euler_bounds[i][1],
        valinit=euler_angles[key],
    )

    slider_lst.append(euler_slider)

    # The function to be called anytime a slider's value changes
    def update_factory(key):
        def update(val):
            global recip_latt
            euler_angles[key] = val
            plot_qs = euler_rotation(all_qs, *euler_angles.values())
            recip_latt.remove()
            recip_latt = ax.scatter(*np.asarray(plot_qs).T, c='k', s=all_fs / 50, alpha=0.75)
            fig.canvas.draw_idle()
        return update

    update_lst.append(update_factory(key))
    slider_lst[i].on_changed(update_lst[i])

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')

fig.show()

In [ ]:
test.spots = pd.read_hdf(test.hdf_path, key='xrdmap/reflections/spots')

In [ ]:
dropped_indices = []

for index in test.spots.index:
    spot = test.spots.loc[index]
    if spot['fit_amp'] - spot['fit_offset'] < 0.5:
        dropped_indices.append(index)

In [ ]:
test.spots = test.spots.drop(index = dropped_indices)

In [ ]:
from xrdmaptools.crystal.orientation import iterative_dictionary_indexing

pixel_indices = 2, 29
pixel_df = test.spots[(test.spots['map_x'] == pixel_indices[0])
                        & (test.spots['map_y'] == pixel_indices[1])].copy()
spot_qs = pixel_df[['qx', 'qy', 'qz']].values

fit_ori, fit_min = iterative_dictionary_indexing(spot_qs, test.phases['stibnite'], [np.min(test.tth_arr), np.max(test.tth_arr)],
                                                 cut_off=0.05,
                                                 start_angle=20,
                                                 angle_resolution=0.1, 
                                                 euler_bounds=[[0, 180], [0, 90], [0, 180]])

all_hkls, all_qs, all_fs = generate_reciprocal_lattice(test.phases['stibnite'], tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)))

In [ ]:
skip = 500

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

plot_qs = euler_rotation(all_qs, *fit_ori[-2])

ax.scatter(*np.asarray(plot_qs).T, c='k', s=all_fs / 50, alpha=0.75)
ax.scatter(0, 0, 0, c='k', s=10)

# Plot full Ewald sphere
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
radius = 2 * np.pi / test.wavelength
x =  radius * np.outer(np.cos(u), np.sin(v))
y = radius * np.outer(np.sin(u), np.sin(v))
z = radius * np.outer(np.ones(np.size(u)), np.cos(v))
#ax.plot_surface(x, y, z - radius, alpha=0.2, color='k', label='Ewald sphere')

q = get_q_vect(test.tth_arr, test.chi_arr, wavelength=test.wavelength)

if pixel_indices is not None:
    pixel_df = test.spots[(test.spots['map_x'] == pixel_indices[0])
                            & (test.spots['map_y'] == pixel_indices[1])].copy()


if pixel_indices is not None:
    ax.scatter(*pixel_df[['qx', 'qy', 'qz']].values.T, s=1, c='r', label='spots')

# Sample geometry
ax.quiver([0, 0], [0, 0], [-2 * radius, -radius], [0, 0], [0, 0], [radius, radius], colors='k')
ax.scatter(0, 0, 0, marker='o', s=10, facecolors='none', edgecolors='k', label='transmission')
ax.scatter(0, 0, -radius, marker='h', s=10, c='b', label='sample')

# Plot sampled Ewald sphere
q_mask = q[:, test.map.mask]
ax.plot_trisurf(q_mask[0].ravel()[::skip],
                q_mask[1].ravel()[::skip],
                q_mask[2].ravel()[::skip],
                alpha=0.5, label='detector')

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')

fig.show()

In [ ]:
test.plot_image([2, 28], spots=True)

In [ ]:
# Informed dictionary indexing

from scipy.spatial.transform import Rotation
from sklearn.metrics.pairwise import euclidean_distances
from xrdmaptools.crystal.orientation import iterative_dictionary_indexing, euler_rotation

indices = 11, 17

phase = test.phases['stibnite']
near_thresh = 0.01
tth_range = [np.min(test.tth_arr), np.max(test.tth_arr)]
cut_off = 0.05
start_angle = 20
angle_resolution = 0.1 
euler_bounds = [[0, 180], [0, 90], [0, 180]]

# Get useful information out of pixel_df
# Does not condense spots...
pixel_df = test.spots[(test.spots['map_x'] == indices[0])
                        & (test.spots['map_y'] == indices[1])].copy()
pixel_df.dropna(inplace=True)

all_hkls, all_qs, all_fs = generate_reciprocal_lattice(test.phases['stibnite'], tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)))
all_qs = np.asarray(all_qs)

dist = euclidean_distances(all_qs)
min_q = np.min(dist[dist > 0])

# Conditionals to avoid single spot from one grain
if len(pixel_df) < 2:
    raise ValueError()

dist = euclidean_distances(pixel_df[['qx', 'qy', 'qz']].values)
if np.max(dist) < min_q:
    raise ValueError()


spot_qs = pixel_df[['qx', 'qy', 'qz']].values
spot_ints = pixel_df[['guess_int']].values

# find potential matces within a threshold
phase_q_vals = phase.reflections['q']
spot_q_vals = np.linalg.norm(spot_qs, axis=1)
diff_arr = np.abs(spot_q_vals[:, np.newaxis] - phase_q_vals[np.newaxis, :])
pot_match = diff_arr < near_thresh
iso_spots = np.sum(pot_match, axis=1) == 1

#iso_spots = -1

# Find rotation matrix just good enough for indexing
# Explicit solution
if np.sum(iso_spots) > 1:
    print('Explicit solution...')
    #max_iso = np.max(spot_ints[iso_spots])
    #spot_ind = np.squeeze(spot_ints == max_iso)
    q_meas = spot_qs[iso_spots]
    #q_calc = phase.Q(phase.reflections['hkl'][pot_match[iso_spots][0]])
    q_calc = phase.Q([phase.reflections['hkl'][ind][0] for ind in pot_match[iso_spots]])
    # Fit in q-space
    rot, rssd = Rotation.align_vectors(q_meas, q_calc)

    if rssd > 1:
        print(f'Explicit fitting exceeded error threshold ({rssd:.3f}). Downgrading method...')
        max_iso = np.max(spot_ints[iso_spots])
        iso_spots = np.squeeze(spot_ints == max_iso)
    else:
        best_orientation = rot.as_matrix()

# Iterate around a single axis of rotation
if np.sum(iso_spots) == 1:
    print('Dictionary indexing around a reflection...')
    q_meas = spot_qs[iso_spots]
    print(f"Rotation center at {pixel_df['img_x'][iso_spots].values[0]}, {pixel_df['img_y'][iso_spots].values[0]}")
    #q_calc = phase.Q(phase.reflections['hkl'][pot_match[iso_spots][0]])
    q_calc = phase.Q([phase.reflections['hkl'][ind][0] for ind in pot_match[iso_spots]])
    # Fit in q-space
    rot_init, rssd = Rotation.align_vectors(q_calc, q_meas)
    norm_q = np.squeeze(q_meas / np.linalg.norm(q_meas))

    ITERATE = True
    # The upper angle may be set by the large symmetric rotation of the point group
    # The step is reduced since there are few iterations for the linear problem 
    angles = np.arange(0, 180, start_angle / 4)
    step = start_angle
    while ITERATE:
        step /= 2 # half the resolution each time
        if step <= angle_resolution:
            ITERATE = False

        # extra rotation about initial rotation
        ext_rot = Rotation.from_rotvec(norm_q[np.newaxis, :]* angles[:, np.newaxis],
                               degrees=True).as_matrix()
        #tot_rot = rot_init.as_matrix() @ ext_rot
        tot_rot = ext_rot @ rot_init.as_matrix()

        min_list = []
        for R_mat in tot_rot:
            dist = euclidean_distances(spot_qs, all_qs @ R_mat)
            min_list.append(np.sum(np.min(dist, axis=1)**2))
        
        min_mask = min_list < cut_off * (np.max(min_list) - np.min(min_list)) + np.min(min_list)
        best_angles = np.asarray(angles)[min_mask]
        best_orientation = tot_rot[np.argmin(min_list)] # this is the output!

        new_angles = []
        for angle in best_angles:
            new_angles.append(angle - step)
            new_angles.append(angle)
            new_angles.append(angle + step)

        angles = np.unique(new_angles)
    
    if np.min(min_list) > 0.5:
        print(f'Axial rotation fitting exceeded error threshold ({np.min(min_list):.3f}). Downgrading method...')
        iso_spots = -1

# No clue. Iterate around mesh of orientations
if np.sum(iso_spots) < 1:
    print('Full iterative dictionary indexing...')
    fit_ori, fit_min = iterative_dictionary_indexing(spot_qs,
                                                     phase,
                                                     tth_range,
                                                     cut_off=cut_off,
                                                     start_angle=start_angle,
                                                     angle_resolution=angle_resolution, 
                                                     euler_bounds=euler_bounds)
    
    I = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
    best_orientation = euler_rotation(I, *fit_ori[-1])
    print(f'Completed with ({fit_min[-1]:.3f}).')


# find next largest iso_spot and try fit explicit rotation matrix...

# generate rotations about meas_hkl vector...


# refine rotation matrix based on indexing values...

Explicit solution...
Explicit fitting exceeded error threshold (1.580). Downgrading method...
Dictionary indexing around a reflection...
Rotation center at 427, 644
Axial rotation fitting exceeded error threshold (0.996). Downgrading method...
Full iterative dictionary indexing...
Completed with (0.152).


In [ ]:
skip = 500
pixel_indices = indices

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

plot_qs = all_qs @ best_orientation

ax.scatter(*np.asarray(plot_qs).T, c='k', s=all_fs / 50, alpha=0.75)
ax.scatter(0, 0, 0, c='k', s=10)

# Plot full Ewald sphere
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
radius = 2 * np.pi / test.wavelength
x =  radius * np.outer(np.cos(u), np.sin(v))
y = radius * np.outer(np.sin(u), np.sin(v))
z = radius * np.outer(np.ones(np.size(u)), np.cos(v))
#ax.plot_surface(x, y, z - radius, alpha=0.2, color='k', label='Ewald sphere')

q = get_q_vect(test.tth_arr, test.chi_arr, wavelength=test.wavelength)

if pixel_indices is not None:
    pixel_df = test.spots[(test.spots['map_x'] == pixel_indices[0])
                            & (test.spots['map_y'] == pixel_indices[1])].copy()

if pixel_indices is not None:
    ax.scatter(*pixel_df[['qx', 'qy', 'qz']].values.T, s=1, c='r', label='spots')

# Sample geometry
ax.quiver([0, 0], [0, 0], [-2 * radius, -radius], [0, 0], [0, 0], [radius, radius], colors='k')
ax.scatter(0, 0, 0, marker='o', s=10, facecolors='none', edgecolors='k', label='transmission')
ax.scatter(0, 0, -radius, marker='h', s=10, c='b', label='sample')

# Plot sampled Ewald sphere
q_mask = q[:, test.map.mask]
ax.plot_trisurf(q_mask[0].ravel()[::skip],
                q_mask[1].ravel()[::skip],
                q_mask[2].ravel()[::skip],
                alpha=0.5, label='detector')

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')

fig.show()

In [ ]:
test.phases['stibnite'].get_hkl_reflections(tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)))

In [ ]:
# Explicit indexing

from scipy.spatial.transform import Rotation
from sklearn.metrics.pairwise import euclidean_distances
from xrdmaptools.crystal.orientation import euler_rotation
from xrdmaptools.crystal.Phase import generate_reciprocal_lattice
from xrdmaptools.reflections.spot_blob_indexing import get_q_vect
from itertools import product
from tqdm import tqdm

indices = (81, 167)

phase = test.phases['LiNbO3']
near_thresh = 0.1
required_spots = 2

# Get useful information out of pixel_df
# Does not condense spots...
pixel_df = test.spots[(test.spots['map_x'] == indices[0])
                        & (test.spots['map_y'] == indices[1])].copy()
pixel_df.dropna(inplace=True)

all_hkls, all_qs, all_fs = generate_reciprocal_lattice(phase, tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)))
all_qs = np.asarray(all_qs)

dist = euclidean_distances(all_qs)
min_q = np.min(dist[dist > 0])

# Conditionals to avoid single spot from one grain
if len(pixel_df) < 2:
    raise ValueError()

dist = euclidean_distances(pixel_df[['qx', 'qy', 'qz']].values[:required_spots])
if np.max(dist) < min_q:
    raise ValueError()


spot_qs = pixel_df[['qx', 'qy', 'qz']].values
spot_ints = pixel_df[['guess_int']].values

# Sort spots from largest to smallest intensity
zipped = list(zip(spot_ints, spot_qs))
zipped.sort(reverse=True)
spot_ints, spot_qs = zip(*zipped)


# find potential matces within a threshold
phase_q_vals = phase.reflections['q']
spot_q_vals = np.linalg.norm(spot_qs, axis=1)
diff_arr = np.abs(spot_q_vals[:, np.newaxis] - phase_q_vals[np.newaxis, :])
pot_match = diff_arr < near_thresh

phase_mask = np.any(pot_match, axis=1)
pot_match = pot_match[phase_mask]
spot_qs = np.array(spot_qs)[phase_mask]
#spot_int = spot_ints[phase_mask]

fit_q_meas = spot_qs[:required_spots]
fit_q_calc = []
for i, pot_match_i in enumerate(pot_match[:required_spots]):
    match_ind = np.where(pot_match_i)[0]
    hkls = [phase.reflections['hkl'][ind] for ind in match_ind]

    # Add equivalent potential indices. Only first index restricted
    # Not sure how much this actually helps...
    if i > 0:
        ext_hkls = []
        [ext_hkls.extend(list(phase.lattice.equivalent_hkls(hkl))) for hkl in hkls];
        hkls = ext_hkls

    q_calc = phase.Q(hkls)
    fit_q_calc.append(q_calc)


if len(fit_q_calc) > 0:
    print('first')
    combos = list(product(*fit_q_calc))

    rot_list, rssd_list = [], []
    for q_calc in combos:
        rot, rssd = Rotation.align_vectors(q_calc, fit_q_meas)
        rot_list.append(rot.as_matrix())
        rssd_list.append(rssd)

    min_ind = np.argmin(rssd_list)
    best_orientation = rot_list[min_ind]

else:
    min_ind = np.nan
    best_orientation = np.nan
        
print(f'Best orientation found at {min_ind} with rssd of {rssd_list[min_ind]:.3f}')

for combo in combos[min_ind][1:]:
    if phase.HKL(combo) not in phase.reflections['hkl']:
        print('Equivalent hkl used successfully. The extra computation was worth it!')
        break

first
Best orientation found at 12 with rssd of 0.285


In [ ]:
from scipy.spatial.transform import Rotation
from itertools import product

def explicit_indexing(spot_qs, # Must already be sorted...
                      phase,
                      near_thresh=0.05,
                      required_spots=2,
                      check_equivalent=True):

    # Sort spots from largest to smallest intensity
    #zipped = list(zip(spot_ints, spot_qs))
    #zipped.sort(reverse=True)
    #spot_ints, spot_qs = zip(*zipped)

    # find potential matces within a threshold
    phase_q_vals = phase.reflections['q']
    spot_q_vals = np.linalg.norm(spot_qs, axis=1)
    diff_arr = np.abs(spot_q_vals[:, np.newaxis] - phase_q_vals[np.newaxis, :])
    pot_match = diff_arr < near_thresh

    # Distinguish by phase kind of
    phase_mask = np.any(pot_match, axis=1)
    pot_match = pot_match[phase_mask]
    spot_qs = np.array(spot_qs)[phase_mask]
    #spot_int = spot_ints[phase_mask]

    fit_q_meas = spot_qs[:required_spots]
    fit_q_calc = []
    for i, pot_match_i in enumerate(pot_match[:required_spots]):
        match_ind = np.where(pot_match_i)[0]
        hkls = [phase.reflections['hkl'][ind] for ind in match_ind]

        # Add equivalent potential indices. Only first index restricted
        # Not sure how much this actually helps...
        if check_equivalent and i > 0:
            ext_hkls = []
            [ext_hkls.extend(list(phase.lattice.equivalent_hkls(hkl))) for hkl in hkls];
            hkls = ext_hkls

        q_calc = phase.Q(hkls)
        fit_q_calc.append(q_calc)

    if len(fit_q_calc) > 0:
        combos = list(product(*fit_q_calc))

        rot_list, rssd_list = [], []
        for q_calc in combos:
            rot, rssd = Rotation.align_vectors(q_calc, fit_q_meas)
            rot_list.append(rot.as_matrix())
            rssd_list.append(rssd)

        min_ind = np.argmin(rssd_list)
        min_rssd = rssd_list[min_ind]
        best_orientation = rot_list[min_ind]

    else:
        min_rssd = np.nan
        best_orientation = np.nan
    #print(f'Best orientation found at {min_ind} with rssd of {rssd_list[min_ind]:.3f}')

    # This was worth it!
    #for combo in combos[min_ind][1:]:
    #    if phase.HKL(combo) not in phase.reflections['hkl']:
    #        print('Equivalent hkl used successfully. The extra computation was worth it!')
    #        break

    return best_orientation, min_rssd

In [ ]:
skip = 500
pixel_indices = indices

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

#plot_qs = all_qs @ rot_list[np.argmin(rssd_list)]
plot_qs = all_qs @ best_orientation

ax.scatter(*np.asarray(plot_qs).T, c='k', s=all_fs / 50, alpha=0.75)
ax.scatter(0, 0, 0, c='k', s=10)

# Plot full Ewald sphere
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
radius = 2 * np.pi / test.wavelength
x =  radius * np.outer(np.cos(u), np.sin(v))
y = radius * np.outer(np.sin(u), np.sin(v))
z = radius * np.outer(np.ones(np.size(u)), np.cos(v))
#ax.plot_surface(x, y, z - radius, alpha=0.2, color='k', label='Ewald sphere')

q = get_q_vect(test.tth_arr, test.chi_arr, wavelength=test.wavelength)

if pixel_indices is not None:
    pixel_df = test.spots[(test.spots['map_x'] == pixel_indices[0])
                            & (test.spots['map_y'] == pixel_indices[1])].copy()

if pixel_indices is not None:
    ax.scatter(*pixel_df[['qx', 'qy', 'qz']].values.T, s=1, c='r', label='spots')

# Sample geometry
ax.quiver([0, 0], [0, 0], [-2 * radius, -radius], [0, 0], [0, 0], [radius, radius], colors='k')
ax.scatter(0, 0, 0, marker='o', s=10, facecolors='none', edgecolors='k', label='transmission')
ax.scatter(0, 0, -radius, marker='h', s=10, c='b', label='sample')

# Plot sampled Ewald sphere
q_mask = q[:, test.map.mask]
ax.plot_trisurf(q_mask[0].ravel()[::skip],
                q_mask[1].ravel()[::skip],
                q_mask[2].ravel()[::skip],
                alpha=0.5, label='detector')

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')

fig.show()

In [ ]:
I = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

euler_mat = euler_rotation(I, 10, 10, 10)
Rotation.from_matrix(euler_mat).as_euler('zxz', degrees=True)

array([10., 10., 10.])

In [ ]:
# Explicit indexing across full map

from sklearn.metrics.pairwise import euclidean_distances
from tqdm import tqdm
from xrdmaptools.utilities.utilities import combine_nearby_spots
from xrdmaptools.crystal.Phase import generate_reciprocal_lattice
from itertools import product
from scipy.spatial.transform import Rotation

required_spots = 4
phase = test.phases['stibnite']

if phase.reflections is None:
    phase.get_hkl_reflections(tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)))

all_hkls, all_qs, all_fs = generate_reciprocal_lattice(phase, tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)))

dist = euclidean_distances(all_qs)
min_q = np.min(dist[dist > 0])

euler_map = np.empty((*test.map.map_shape, 3))
fit_map = np.empty(test.map.map_shape)

for index in tqdm(range(test.map.num_images)):
    indices = np.unravel_index(index, test.map.map_shape)
    phi1, PHI, phi2 = np.nan, np.nan, np.nan

    pixel_df = test.spots[(test.spots['map_x'] == indices[0])
                        & (test.spots['map_y'] == indices[1])].copy()
    pixel_df.dropna(inplace=True)
    
    # Conditional to avoid too few spots for theoretical indexing
    if len(pixel_df) < required_spots:
        euler_map[indices] = phi1, PHI, phi2
        continue
    
    spot_qs = pixel_df[['qx', 'qy', 'qz']].values
    spot_ints = pixel_df[['fit_integrated']].values
    #spot_chis = pixel_df[['fit_chi0']].values

    spot_qs, spot_ints = combine_nearby_spots(spot_qs, spot_ints, max_dist=0.1, max_neighbors=np.inf)

    # Sort spots from largest to smallest intensity
    zipped = list(zip(spot_ints, spot_qs))
    zipped.sort(reverse=True)
    spot_ints, spot_qs = zip(*zipped)
    spot_ints = list(spot_ints)
    spot_qs = list(spot_qs)

    dist_too_small = True
    while dist_too_small:
        if len(spot_qs) < required_spots:
            euler_map[indices] = phi1, PHI, phi2
            dist_too_small = False
            break
        # Conditional to avoid single spot from one blob
        dist = euclidean_distances(spot_qs)
        
        if dist[0, 1] < min_q:
            del spot_qs[1], spot_ints[1]
        else:
            dist_too_small = False

    if len(spot_qs) < required_spots:
        euler_map[indices] = phi1, PHI, phi2
        continue
    
    best_orientation, min_rssd = explicit_indexing(spot_qs,
                                                   phase,
                                                   near_thresh=0.05,
                                                   required_spots=required_spots,
                                                   check_equivalent=False)
    
    if not np.any(np.isnan(best_orientation)):
        best_euler = Rotation.from_matrix(best_orientation).as_euler('zxz', degrees=True)
        
        euler_map[indices] = best_euler
        fit_map[indices] = min_rssd
    else:
        euler_map[indices] = np.nan, np.nan, np.nan

 16%|█▌        | 525/3381 [00:10<01:18, 36.17it/s] C:\Users\emusterma\AppData\Local\Temp\ipykernel_5492\2468468959.py:48: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rot, rssd = Rotation.align_vectors(q_calc, fit_q_meas)
 33%|███▎      | 1131/3381 [00:36<00:48, 46.38it/s]C:\Users\emusterma\AppData\Local\Temp\ipykernel_5492\2468468959.py:48: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rot, rssd = Rotation.align_vectors(q_calc, fit_q_meas)
 44%|████▎     | 1472/3381 [00:59<01:23, 22.75it/s] C:\Users\emusterma\AppData\Local\Temp\ipykernel_5492\2468468959.py:48: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rot, rssd = Rotation.align_vectors(q_calc, fit_q_meas)
C:\Users\emusterma\AppData\Local\Temp\ipykernel_5492\2468468959.py:48: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rot, rssd

In [ ]:
orientation_map = np.empty((*test.map.map_shape, 3, 3))

for index in range(test.map.num_images):
    indices = np.unravel_index(index, test.map.map_shape)
    
    if np.any(np.isnan(euler_map[indices])):
        orientation_map[indices] = np.nan
    else:
        orientation_map[indices] = Rotation.from_euler('zxz', euler_map[indices], degrees=True).as_matrix()

In [ ]:
test.plot_map(np.abs(orientation_map[:, :, 2, 2]))

In [ ]:
# Iterative dictionary indexing across full map

from sklearn.metrics.pairwise import euclidean_distances
from tqdm import tqdm
from xrdmaptools.reflections.spot_blob_indexing import iterative_dictionary_indexing
from xrdmaptools.crystal.Phase import generate_reciprocal_lattice

all_hkls, all_qs, all_fs = generate_reciprocal_lattice(test.phases['stibnite'], tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)))

dist = euclidean_distances(all_qs)
min_q = np.min(dist[dist > 0])

euler_map = np.empty((*test.map.map_shape, 3))
fit_map = np.empty(test.map.map_shape)

for index in tqdm(range(test.map.num_images)):
    indices = np.unravel_index(index, test.map.map_shape)
    phi1, PHI, phi2 = np.nan, np.nan, np.nan

    pixel_df = test.spots[(test.spots['map_x'] == indices[0])
                        & (test.spots['map_y'] == indices[1])].copy()
    pixel_df.dropna(inplace=True)
    spot_qs = pixel_df[['qx', 'qy', 'qz']].values

    # Conditional to avoid too few spots for theoretical indexing
    if len(pixel_df) < 3:
        euler_map[indices] = phi1, PHI, phi2
        continue
    
    # Conditional to avoid single spot from one grain
    dist = euclidean_distances(pixel_df[['qx', 'qy', 'qz']].values)
    if np.max(dist) < min_q:
        euler_map[indices] = phi1, PHI, phi2
        continue
    
    fit_ori, fit_min = iterative_dictionary_indexing(spot_qs, test.phases['stibnite'], [np.min(test.tth_arr), np.max(test.tth_arr)],
                                                 cut_off=0.05,
                                                 start_angle=20,
                                                 angle_resolution=1, 
                                                 euler_bounds=[[0, 180], [0, 90], [0, 180]])
    
    euler_map[indices] = fit_ori[-1]
    fit_map[indices] = fit_min[-1]

100%|██████████| 3381/3381 [2:06:57<00:00,  2.25s/it]  


In [ ]:
from xrdmaptools.crystal.orientation import g_func
g = g_func(*euler_map.reshape(test.map.num_images, 3).T)

In [ ]:
ipole = [0, 1, 0]

colors, r, theta = [], [], []
for i in range(len(g)):
    
    # Take only part of the orientation matrix
    V = np.dot(g[i], ipole) # order is g, then ipole

    # Determine pixel color according to unit triangle
    R, G, B = 0, 0, 0
    base = 30 / 255 #all pixels are a bit brighter
    R=np.abs(V[2]) + base # referenced to crystallographic axes, but should be able to reference any axis
    G=np.abs(V[0]) + base
    B=np.abs(V[1]) + base

    # Normalize and brighten colors
    max_c = np.max([R,G,B])
    R = R / max_c
    G = G / max_c
    B = B / max_c
    
    # Setting pixel transparency if called
    A=1
    colors.append([R, G, B, A])

colors = np.asarray(colors)
colors = colors.reshape((*test.map.map_shape, 4))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=200)

ax.imshow(colors)
ax.set_title(str(ipole))

fig.show()

In [ ]:
test.plot_interactive_map(display_map=colors, img_vmin=0, img_vmax=0.1)

In [ ]:
test.plot_image([7, 46])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=200)

fit_map[fit_map < 0.0001] = np.nan

ax.imshow(fit_map)
ax.set_title('Fit')

fig.show()

In [ ]:
norm_fit_map = fit_map.copy()

for index in range(test.map.num_images):
    indices = np.unravel_index(index, test.map.map_shape)
    
    pixel_df = test.spots[(test.spots['map_x'] == indices[0])
                        & (test.spots['map_y'] == indices[1])].copy()
    pixel_df.dropna(inplace=True)

    # Conditional to avoid too few spots for theoretical indexing
    if len(pixel_df) < 3:
        euler_map[indices] = phi1, PHI, phi2
        continue
    
    # Conditional to avoid single spot from one grain
    dist = euclidean_distances(pixel_df[['qx', 'qy', 'qz']].values)
    if np.max(dist) < min_q:
        euler_map[indices] = phi1, PHI, phi2
        continue

    norm_fit_map[indices] /= len(pixel_df)

fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=200)

im = ax.imshow(norm_fit_map)
fig.colorbar(im, ax=ax)

fig.show()

In [ ]:
from xrdmaptools.utilities.image_corrections import rescale_array

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=200)

trans_colors = colors.copy()
trans_colors[:, :, -1] = -rescale_array(norm_fit_map, lower=-1, upper=0)
mask = trans_colors[:, :, -1] == colors[:, :, -1]
trans_colors[mask, -1] = 0

ax.imshow(trans_colors)
ax.set_title(str(ipole))
ax.set_facecolor('black')

fig.show()

In [ ]:
test.plot_interactive_map(display_map=colors, img_vmin=0, img_vmax=0.1)

In [ ]:
#from xrdmaptools.utilities.utilities import label_nearest_spots
from xrdmaptools.geometry.geometry import *

indices = 9, 75

pixel_df = test.pixel_spots(indices).copy()
pixel_df.dropna(inplace=True)
q_spots = pixel_df[['qx', 'qy', 'qz']].values
polar_spots = pixel_df[['fit_tth0', 'fit_chi0']].values
spot_ints = pixel_df[['guess_int']].values


out = label_nearest_spots(q_spots, max_dist=0.1, max_neighbors=np.inf)
new_out = np.empty((len(out), 3))
new_out[:, -1] = out[:, -1]
new_out[:, :2] = q_2_polar(out, wavelength=test.wavelength)
out = new_out

In [ ]:
from xrdmaptools.utilities.utilities import arbitrary_center_of_mass

def combine_nearby_spots(spots, *weights, max_dist, max_neighbors=np.inf):
    # Spots are weighted by the first eight!
    
    labeled_spots = label_nearest_spots(spots, max_dist=max_dist, max_neighbors=max_neighbors)

    combined_spots = []
    combined_weights = []
    for label in np.unique(labeled_spots[:, -1]):
        label_mask = labeled_spots[:, -1] == label
        #combined_spot = np.mean(labeled_spots[label_mask][:, :-1], axis=0)
        combined_spot = arbitrary_center_of_mass(np.squeeze(np.asarray(weights)[0])[label_mask],
                                                  *labeled_spots[label_mask][:, :-1].T)
        combined_spots.append(combined_spot)
        for weight in weights:
            combined_weight = np.sum(weight[label_mask])
            combined_weights.append(combined_weight)

    return combined_spots, combined_weights

In [ ]:
out, weights = combine_nearby_spots(q_spots, spot_ints, max_dist=0.1, max_neighbors=np.inf)
new_out = np.empty((len(out), 3))
new_out[:, :2] = q_2_polar(out, wavelength=test.wavelength)
out = new_out

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(test.map.images[indices], vmin=0, vmax=0.1)
fig.colorbar(im, ax=ax)

ax.scatter(*estimate_image_coords(out[:, :-1], test.tth_arr, test.chi_arr).T,
           #c=out[:, -1],
           c=weights,
           s=1,
           cmap='jet')

fig.show()